<a href="https://colab.research.google.com/github/Thandeka20/High-spatial-resolution-imagery-vs-high-spectral-imagery/blob/main/Code/Sentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='thandeka-skosana')

In [5]:
#import modules
!pip install rasterio
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import re
import os
from IPython.display import Image, display
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**save_df_to_drive**
Function that saves a Pandas DataFrame as a csv file to a Google Drive folder specified by a google drive file path

In [6]:

def save_df_to_drive(df, file_path_in_drive):
    """
    Function that saves a Pandas DataFrame as a csv file to a Google Drive folder specified by a Google Drive file path

    parameters:
      df (pd.DataFrame): The Pandas DataFrame needing to be saved
      file_path_in_drive (str): Google Drive folder file path that will store the CSV file
    """
    try:
        # Ensure the destination directory exists
        drive.mount('/content/drive')  # Mount Google Drive to access files

        # Get the absolute path in Google Drive
        absolute_path_in_drive = os.path.join('/content/drive/My Drive', file_path_in_drive)

        # Ensure the destination directory exists in Google Drive
        os.makedirs(os.path.dirname(absolute_path_in_drive), exist_ok=True)

        # Save the DataFrame to the specified file path in Google Drive
        df.to_csv(absolute_path_in_drive, index=False)

        print(f"DataFrame saved to Google Drive at '{absolute_path_in_drive}'")

    except Exception as e:
        print("An error occurred:", str(e))

# Example usage:
# Assuming your file_path_in_drive is 'YourFolder/YourFile.csv'
# Make sure to change it according to your actual folder structure
#save_df_to_drive(your_dataframe, 'YourFolder/YourFile.csv')


**Image collection**
get_sentinel_images_ee
Function that allows the download of Sentinel-2 images from GEE

In [ ]:
#import ee
#from datetime import datetime

#def get_sentinel_images_ee(latitude, longitude, start_date, end_date):
    #"""
    #Function to search and download Sentinel-2 images using Google Earth Engine.

    #Parameters:
        #latitude (float): Latitude coordinate of the desired location.
        #longitude (float): Longitude coordinate of the desired location.
        #start_date (str): Start date of the date range in the format 'YYYY-MM-DD'.
        #end_date (str): End date of the date range in the format 'YYYY-MM-DD'.

    #Returns:
        #List of image collection IDs for the retrieved Sentinel-2 images.
    #"""
    # Create a point geometry representing the location
    #point = ee.Geometry.Point(longitude, latitude)

    # Define the date range
    #date_range = ee.DateRange(datetime.strptime(start_date, '%Y-%m-%d'), datetime.strptime(end_date, '%Y-%m-%d'))

    # Filter Sentinel-2 data based on location and date range
    #collection = (ee.ImageCollection('COPERNICUS/S2')
                  #.filterBounds(point)
                  #.filterDate(date_range))

    # Get a list of image IDs in the collection
    #image_ids = collection.aggregate_array('system:id').getInfo()

    #return image_ids

# Example usage:
#latitude = -25.12  # Hazyview
#longitude = 30.88  # Hazyview
#start_date = '2023-07-27'
#end_date = '2023-07-28'

#result = get_sentinel_images_ee(latitude, longitude, start_date, end_date)
#print("Image Collection IDs:", result)

In [7]:
#visualize the Sentinel-2 scene
# Define the bands to use for visualization
bands = ee.List(["B4", "B3", "B2"])

# Define visualization parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}

# Example: Display an image using the defined visualization parameters
image = ee.Image('COPERNICUS/S2/20230727T073621_20230727T075808_T36JTT')
display_image = Image(url=image.visualize(**vis_params).getThumbURL({'dimensions': '500x500'}))
display(display_image)

**Calculating indices**

In [9]:
# Define vegetation indices
GI = image.expression('(GREEN)/(RED)', {
    'GREEN': image.select('B3'),
    'RED': image.select('B4'),
}).rename('GI')

IRG = image.expression('(RED-GREEN)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
}).rename('IRG')

NGRDI = image.expression('(GREEN-RED)/(GREEN+RED)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
}).rename('NGRDI')

VARI = image.expression('(GREEN-RED)/(GREEN+RED+BLUE)', {
    'RED': image.select('B3'),
    'GREEN': image.select('B4'),
    'BLUE': image.select('B2'),
}).rename('VARI')

VDVI = image.expression('(2*GREEN-RED-BLUE)/(2*GREEN+RED+BLUE)', {
    'RED': image.select('B4'),
    'GREEN': image.select('B3'),
    'BLUE': image.select('B2'),
}).rename('VDVI')

WBI = image.expression('(Blue - Red) / (Blue + Red)', {
    'Blue': image.select('B2'),
    'Red': image.select('B4'),
}).rename('WBI')

# Add computed variables as bands to the image_final
image_final = image.addBands([GI, IRG, NGRDI, VARI, VDVI, WBI])

# Print image bands
print(image_final.getInfo(), 'image_bands')

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [1830, 1830], 'crs': 'EPSG:32736', 'crs_transform': [60, 0, 199980, 0, -60, 7300000]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [10980, 10980], 'crs': 'EPSG:32736', 'crs_transform': [10, 0, 199980, 0, -10, 7300000]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [5490, 5490], 'crs': 'EPSG:32736', 'crs_transform': [20, 0, 199980, 0, -20, 7300000

In [17]:
# Specify the Google Drive file ID of the shapefile
file_id = 'https://drive.google.com/drive/folders/1GY1IZ115x7zOM6WTXpHS2hNF_wHslpfI'
output_path = '/content/drive/My Drive/Colab Notebooks/Data/TrainingSet_SabieCroc/Training_setSc.shp'

#loadshapefile using geopandas
training_data = gpd.read_file(output_path)

# Display the GeoDataFrame
print(training_data.head())



                                    Name  \
0  0e9fd6a4c5704984a97731f76c9fdc86.jpeg   
1  104fe7223f3543ddbbb304faaac44eaf.jpeg   
2  12c5a6fe83ed45d388722f623b9dfc7c.jpeg   
3  158cb3a73f824c7da590cb5dacbfaa7c.jpeg   
4  4105e29b7b9743539045ff4992cb673d.jpeg   

                                                Path  GPS_ID   ID LULC_Class  \
0  C:\Users\Liam\Desktop\Sabie_Croc\Day_10\0e9fd6...    3773  2.0  Alien_Gum   
1  C:\Users\Liam\Desktop\Sabie_Croc\Day_10\104fe7...    3656  2.0  Alien_Gum   
2  C:\Users\Liam\Desktop\Sabie_Croc\Day_10\12c5a6...    3670  2.0  Alien_Gum   
3  C:\Users\Liam\Desktop\Sabie_Croc\Day_10\158cb3...    3737  2.0  Alien_Gum   
4  C:\Users\Liam\Desktop\Sabie_Croc\Day_10\4105e2...    3726  2.0  Alien_Gum   

  DIRECTION Done  Distance_t  IAP_Age  IAP_Densit  ... IAP_Dens_1  Notes_1  \
0        SW    Y         2.0     12.0         100  ...       None     None   
1         N    Y        10.0     16.0         100  ...       None     None   
2        SE    Y    

Loading training data from GEE

In [19]:
#select bands
bands= ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B8','B8A', 'B9', 'B11', 'B12','GI', 'IRG', 'NGRDI', 'VARI', 'VDVI', 'WBI',];

# Sample regions from the image at training feature locations
training_data = image_final.select(bands).sampleRegions(
    collection=training_data,
    properties=['ID', 'LULC_Class'],
    scale=10
)

Classification using random forest

In [21]:
#Classification
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training_data,
    classProperty='ID',
    inputProperties=bands
)
classified = image_final.select(bands).classify(classifier)

Defining a colour palette for the classified map

In [22]:
# Define the palette
SabieCrocPalette = [
    '351C75', 'F91DF9', '980A7D', '741b47', 'fd0618', 'E06666', 'ffcc99', 'ffffff', '999999', 'a8a800',
    '6aa84f', '14870e', 'DB992D', 'ff7f00', '000000', '0a14f9', '08f3e4'
]

# Visualization parameters
viz = {
    'min': 1,
    'max': 17,
    'palette': SabieCrocPalette
}

# Create a Map object using geemap
m = geemap.Map(center=[20, 0], zoom=3)

# Add the classified image to the map
m.addLayer(classified, viz, 'Classification')
m.centerObject(classified, zoom=10)

# Display the legend
m.add_legend(title='Classification', colors=SabieCrocPalette, labels=[str(i) for i in range(1, 18)])

# Display the map
m




Map(center=[-24.91471009556816, 30.694320050468605], controls=(WidgetControl(options=['position', 'transparent…